In [1]:
from ctgan import CTGAN
import pandas as pd
import torch
from sdmetrics.reports.single_table import QualityReport
import torch.version
from sdv.single_table.copulagan import CopulaGANSynthesizer
from sdv.single_table import TVAESynthesizer
from sdv.metadata import SingleTableMetadata

In [2]:

print(torch.cuda.is_available())  # Returns True if CUDA is available
print(torch.version.cuda)  # Prints the version of CUDA supported by the installed PyTorch
print(torch.__version__)
print(torch.cuda.get_device_name(0))   

True
12.4
2.5.1
NVIDIA GeForce RTX 2080 Ti


In [3]:
df = pd.read_excel('D:\Ritik\data_science\gan-model\COVID19-web.xlsx')

In [4]:
df.head()

,TrialID,Last Refreshed on,Public title,Scientific title,Acronym,Primary sponsor,Date registration,Date registration3,Export date,Source Register,...,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79
0,CTRI/2024/12/077684,2024-12-09,Assessment of difficult intubation prior to su...,USE OF VIDEOLARYNGOSCOPY AND VIDIAC (VIDEOLARY...,NaN,self,2024-05-12 00:00:00,20241205,2024-12-13 13:18:28,CTRI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NCT06723821,2024-12-09,Noninvasive Neuromodulation in the Management ...,Effectiveness of Noninvasive Neuromodulation i...,NaN,University of Alcala,2024-04-12 00:00:00,20241204,2024-12-13 13:18:28,ClinicalTrials.gov,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NCT06724614,2024-12-09,Efficacy and Safety of VDPHL01 in Males with AGA,"A Randomized, Double-Blind, Placebo-Controlled...",NaN,"Veradermics, Inc.",2024-04-12 00:00:00,20241204,2024-12-13 13:18:28,ClinicalTrials.gov,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ChiCTR2400093398,2024-12-09,Development and application of a smart platfor...,Development and application of a smart platfor...,NaN,Guizhou Nursing Vocational and technical College,2024-04-12 00:00:00,20241204,2024-12-13 13:18:28,ChiCTR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NCT06717321,2024-12-09,Percutaneous Endoscopic Gastrostomy Versus Per...,Percutaneous Endoscopic Gastrostomy Versus Per...,NaN,Jerry Dang,2024-03-12 00:00:00,20241203,2024-12-13 13:18:28,ClinicalTrials.gov,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
print(df.columns)

Index(['TrialID', 'Last Refreshed on', 'Public title', 'Scientific title',
       'Acronym', 'Primary sponsor', 'Date registration', 'Date registration3',
       'Export date', 'Source Register', 'web address', 'Recruitment Status',
       'other records', 'Inclusion agemin', 'Inclusion agemax',
       'Inclusion gender', 'Date enrollement', 'Target size', 'Study type',
       'Study design', 'Phase', 'Countries', 'Contact Firstname',
       'Contact Lastname', 'Contact Address', 'Contact Email', 'Contact Tel',
       'Contact Affiliation', 'Inclusion Criteria', 'Exclusion Criteria',
       'Condition', 'Intervention', 'Primary outcome', 'Secondary outcome',
       'results date posted', 'results date completed', 'results url link',
       'Retrospective flag', 'Bridging flag truefalse', 'Bridged type',
       'results yes no', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43',
       'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47',
       'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 5

In [6]:
df.shape

(22995, 80)

In [7]:
df.corr()

C:\Users\Mobcoder ID-090\AppData\Local\Temp\ipykernel_18964\1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,Unnamed: 65,Unnamed: 74,Unnamed: 76,Unnamed: 78
Unnamed: 65,NaN,NaN,NaN,NaN
Unnamed: 74,NaN,NaN,NaN,NaN
Unnamed: 76,NaN,NaN,NaN,NaN
Unnamed: 78,NaN,NaN,NaN,NaN


In [8]:
df.isnull().sum()

TrialID                  0
Last Refreshed on        0
Public title             2
Scientific title         2
Acronym              18129
                     ...  
Unnamed: 75          22994
Unnamed: 76          22995
Unnamed: 77          22994
Unnamed: 78          22994
Unnamed: 79          22994
Length: 80, dtype: int64

In [9]:
df = df.drop(columns=[ 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43',
                       'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 
                       'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 
                       'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52', 
                       'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55', 
                       'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58', 
                       'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61', 
                       'Unnamed: 62', 'Unnamed: 63', 'Unnamed: 64', 
                       'Unnamed: 65', 'Unnamed: 66', 'Unnamed: 67', 
                       'Unnamed: 68', 'Unnamed: 69', 'Unnamed: 70', 
                       'Unnamed: 71', 'Unnamed: 72', 'Unnamed: 73', 
                       'Unnamed: 74', 'Unnamed: 75', 'Unnamed: 76', 
                       'Unnamed: 77', 'Unnamed: 78', 'Unnamed: 79'], errors='ignore')


In [10]:
df.shape

(22995, 41)

In [11]:
df.head()

,TrialID,Last Refreshed on,Public title,Scientific title,Acronym,Primary sponsor,Date registration,Date registration3,Export date,Source Register,...,Intervention,Primary outcome,Secondary outcome,results date posted,results date completed,results url link,Retrospective flag,Bridging flag truefalse,Bridged type,results yes no
0,CTRI/2024/12/077684,2024-12-09,Assessment of difficult intubation prior to su...,USE OF VIDEOLARYNGOSCOPY AND VIDIAC (VIDEOLARY...,NaN,self,2024-05-12 00:00:00,20241205,2024-12-13 13:18:28,CTRI,...,Intervention1: nil: nil<br>Intervention2: nil:...,To correlate VIDIAC score attained during vide...,To compare VIDIAC score with Simplified Airway...,NaN,NaN,NaN,Yes,False,,NaN
1,NCT06723821,2024-12-09,Noninvasive Neuromodulation in the Management ...,Effectiveness of Noninvasive Neuromodulation i...,NaN,University of Alcala,2024-04-12 00:00:00,20241204,2024-12-13 13:18:28,ClinicalTrials.gov,...,Other: Neuromodulation group;Other: Neuromodul...,Visual Analogue Scale (VAS),DN4 scale;Pittsburg Sleep Quality Index score;...,NaN,NaN,NaN,Yes,False,,NaN
2,NCT06724614,2024-12-09,Efficacy and Safety of VDPHL01 in Males with AGA,"A Randomized, Double-Blind, Placebo-Controlled...",NaN,"Veradermics, Inc.",2024-04-12 00:00:00,20241204,2024-12-13 13:18:28,ClinicalTrials.gov,...,Drug: VDPHL01;Drug: Placebo,Changes in non-vellus Target Area Hair Counts ...,NaN,NaN,NaN,NaN,No,False,,NaN
3,ChiCTR2400093398,2024-12-09,Development and application of a smart platfor...,Development and application of a smart platfor...,NaN,Guizhou Nursing Vocational and technical College,2024-04-12 00:00:00,20241204,2024-12-13 13:18:28,ChiCTR,...,Control group:1. Day 1 to 2<br>Control group: ...,Standard procedure test form for wearing and r...,Satisfaction degree of prevention and control ...,NaN,NaN,NaN,No,False,,NaN
4,NCT06717321,2024-12-09,Percutaneous Endoscopic Gastrostomy Versus Per...,Percutaneous Endoscopic Gastrostomy Versus Per...,NaN,Jerry Dang,2024-03-12 00:00:00,20241203,2024-12-13 13:18:28,ClinicalTrials.gov,...,Procedure: (Arm 1): PEG;Procedure: (Arm 2): PEG-J,Number of Participants Who Developed Post-Proc...,Number of Participants Requiring Additional Pr...,NaN,NaN,NaN,Yes,False,,NaN
